In [ ]:
from yfdata.datahandler import Datahandler
import pandas as pd
import numpy as np
import logging
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
pio.renderers.default = 'notebook'

class ProjectX:
    def __init__(self,file):
        self.file = file
        
    def _prepare_fundamentals(self):
        df = Datahandler(self.file).get_fundamentaldata()
        if df.empty:
            return {}
        
        revenue = df.loc['TotalRevenue']
        netincome = df.loc['NetIncome']
        netmargin = netincome/revenue*100
        
        return {
            'Revenue' : revenue,
            'Net Income' : netincome,
            'Net Margin' : netmargin
            }
    
    def _prepare_pricedata(self):
        df = Datahandler(self.file).get_pricedata()
        if df.empty:
            return {}
        
        _returns = df['Close'].pct_change().fillna(0.0)*100
        
        rolling_beta = _returns.rolling(14).mean().dropna()
        rolling_sigma = _returns.rolling(14).std().dropna()
        return_by_year = pd.Series(dtype='int')
        
        for j, i in df.groupby(df.index.year)['Close']:
            if len(i) <= 240:
                continue
                
            ret = int((i.iloc[-1]/i.iloc[0]-1)*100)
            return_by_year.loc[j] = ret
            
        return {
            'β' : rolling_beta,
            'σ' : rolling_sigma,
            'ROI' : return_by_year
            }
    
    def f(self):
        data = self._prepare_fundamentals()
        
        fig = go.Figure()
        fig.update_layout(
            template='presentation',
            font=dict(size=11,family='montserat')
        )
        
        for key, val in data.items():
            if key != 'Net Margin':
                fig.add_trace(go.Bar(x=val.index,y=val,name=key,hovertemplate='<b>%{x}</b><br>%{y}$'))
            else:
                fig.add_trace(go.Scatter(x=val.index,y=val,name=key,yaxis='y2',hovertemplate='<b>%{x}</b><br>%{y:.2f}%'))
                
        fig.update_layout(
            yaxis=dict(title='$'),
            yaxis2=dict(overlaying='y',side='right',title='%'),
            title=dict(text=f'{stock} Income',font=dict(size=17,family='montserat',weight='bold')),
            barmode='group'
        )
        fig.show()
        
    def p(self):
        data = self._prepare_pricedata()
        
        fig = make_subplots(rows=2,cols=1,x_title='Date',row_heights=[0.65,0.35])
        fig.update_layout(
            template='presentation',
            font=dict(size=11,family='montserat')
        )
        
        for key,val in data.items():
            if key != 'ROI':
                fig.add_trace(go.Scatter(x=val.index,y=val,name=key,hovertemplate='<b>%{x}</b><br>%{y:.2f}%'),row=1,col=1)
            else:
                fig.add_trace(go.Bar(x=val.index,y=val,name=key,hovertemplate='<b>%{x}</b><br>%{y}%'),row=2,col=1)
        
        fig.update_layout(title=dict(text=f'{stock} Returns',font=dict(size=17,family='montserat',weight='bold')))
        fig.show()
        
        
if __name__ == '__main__':
    logging.basicConfig(level=logging.WARNING)
    stock = 'APO'
    obj = ProjectX(stock)
    obj.p()
    obj.f()